Draw a circle with a random centre but fixed radius on a 64x64 grid

In [ ]:
import numpy as np
import math

gridSize = 64
radius = 10

def oneRandomCircle():

    outputGrid = np.zeros( shape=(gridSize,gridSize) )
    x = np.random.uniform( 0.0+radius, gridSize-radius ) # keep the whole circle on the plot for now
    y = np.random.uniform( 0.0+radius, gridSize-radius )
    
    # This method is rubbish, but has the "useful" side-effect of making irregular images
    for i in range(gridSize):
        for j in range(gridSize):
            deltaX = i - x
            deltaY = j - y
            deltaR = deltaX*deltaX + deltaY*deltaY
            if math.fabs( deltaR - radius ) < 2.0:
                outputGrid[j,i] = 1 # I've never understood the indexing but this seems to be right
    
    return outputGrid, (x,y)

Plot a test image

In [ ]:
import matplotlib.pyplot as pl

def plotOneImage( inputArray, centre=None ):
    
    pl.imshow( inputArray, cmap='gray' )
    
    if centre is not None:
        pl.scatter(centre[0], centre[1], color='r')
    
    pl.tight_layout()
    pl.axis('off')
    #pl.xlabel(centre)
    pl.show()

Create datasets to train the CNN to find the centres

In [ ]:
N = 1000
trainInput = np.empty( shape=[N,gridSize,gridSize] )
trainOutput = np.empty( shape=[N,2] ) # N samples, 2 output coordinates

for i in range(N):
    
    testImage, centre = oneRandomCircle()
    
    trainInput[i] = testImage
    trainOutput[i] = centre

plotOneImage(trainInput[5], trainOutput[5])

Now train a CNN to find the middle

Based on https://www.tensorflow.org/tutorials/images/cnn

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Rescaling

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='relu')) # 2 coordinates to output
model.add(Rescaling(gridSize)) # NNs like values 0-1, so let's scale to full grid rather than make it learn

model.compile( loss='mean_squared_error', optimizer='adam' )
model.fit( trainInput, trainOutput, epochs=5, batch_size=10, verbose=2)

Now demonstrate the prediction

In [ ]:
for i in range(5):

    testImage, centre = oneRandomCircle()
    testInput = np.empty( shape=[1,gridSize,gridSize] )
    testInput[0] = testImage

    testOutput = model.predict( testInput )[0] # input and output are both length-1 arrays

    plotOneImage( testImage, testOutput )